In [2]:
!pip install faiss-cpu 
!pip install wikipedia


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Wikipedia Retriever

In [3]:
from langchain_community.retrievers import WikipediaRetriever

wiki = WikipediaRetriever(top_k_results=2,lang="en")

In [4]:
query = "what is latest new of todays in india"

result = wiki.invoke(query)
result

[Document(metadata={'title': '2025 India–Pakistan conflict', 'summary': "The 2025 India–Pakistan conflict was a brief armed conflict between India and Pakistan that began on 7 May 2025, after India launched missile strikes on Pakistan, in a military campaign codenamed Operation Sindoor. India said that the operation was in response to the Pahalgam terrorist attack in Indian-administered Jammu and Kashmir on 22 April 2025 in which 26 civilians were killed. India accused Pakistan of supporting cross-border terrorism, which Pakistan denied.\nOn 7 May, India launched Operation Sindoor with missile strikes on terrorism-related infrastructure facilities of Pakistan-based militant groups Jaish-e-Mohammed and Lashkar-e-Taiba in Pakistan and Pakistan-administered Azad Kashmir, and said that no Pakistani military or civilian facilities were targeted. According to Pakistan, the Indian strikes hit civilian areas, including mosques, and resulted in civilian casualties. Following these strikes, ther

# Vector Store Retriever


In [5]:
from langchain_community.vectorstores import Chroma
import os

from langchain.embeddings import HuggingFaceEmbeddings
huggingfacehub_api_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


C:\Users\ANKIT\AppData\Local\Temp\ipykernel_16708\2258002492.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
f:\study\LangGraph_LangChain\llmenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# Step 1: Your source documents
from langchain_core.documents import Document

documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [8]:
vectorstore = Chroma.from_documents(embedding=embedding_model,collection_name="test",documents=documents)

In [9]:
# convert vectore into retriver

query = "What is Chroma used for"

retriever = vectorstore.as_retriever(search_kwargs={"k":2})


In [11]:
results = retriever.invoke(query)

In [12]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Chroma is a vector database optimized for LLM-based search.

--- Result 2 ---
LangChain helps developers build LLM applications easily.


# MMR

In [13]:
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

# FAISS VECTOR DB

In [14]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(
    documents=documents,
    embedding=embedding_model
)

### Enable the MMR in  the retriever

In [15]:
retriever = vectorstore.as_retriever(
    search_type="mmr",                   # <-- This enables MMR
    search_kwargs={"k": 3, "lambda_mult": 0.5}  # k = top results, lambda_mult = relevance-diversity balance
)

In [17]:
query = "what is langchain"
results = retriever.invoke(query)

In [18]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
LangChain helps developers build LLM applications easily.

--- Result 2 ---
Embeddings convert text into high-dimensional vectors.

--- Result 3 ---
Chroma is a vector database optimized for LLM-based search.


# MultiQuery Retriever